## Exact simulation of the 3/2 model

Different from Baldeaux ([2012](https://doi.org/10.1142/S021902491250032X)), we use an almost exact way to simulate the stock price under the 3/2 model. The dynamics of the stock price under the 3/2 model under the risk-neutral measure are given by

$$
 \frac{dS_t}{S_t} = rdt + \sqrt{V_t}\rho dW_t^1 + \sqrt{V_t}\sqrt{1-\rho^2}dW_t^2 \tag{1}
$$

$$
 dV_t = \kappa V_t (\theta - V_t)dt + \epsilon V_t^{3/2}dW_t^1 \tag{2}
$$

where $W_t^1$ and $W_t^2$ are independent Brownian motions. Regarding the parameters, $r$ represents the constant interest rate, $\rho$ the instantaneous correlation between the return on the stock and the variance process and $\epsilon$ governs the volatility of volatility. The speed of mean reversion is given by $\kappa V_t$ and $\theta$ denotes the long-run mean of the variance process.

Defining $X_t = \frac{1}{V_t}$, we obtain

$$
dX_t = (\kappa + \epsilon^2 - \kappa\theta X_t)dt - \epsilon \sqrt{X_t}dW_t^1 \tag{3}
$$



### Part 1: Almost Exact MC Method:

### Step 1): Simulating $X_T$

$X_T$ is distributed as a noncentral $ \chi^2 $ distribution

$$
\frac{X_T{\rm exp}\lbrace \kappa \theta T \rbrace}{c(T)} \sim \chi^2(\delta, \alpha)
$$

where
$$
\delta = \frac{4(\kappa + \epsilon^2)}{\epsilon^2}, \quad \alpha = \frac{X_0}{c(T)}, \quad c(T) = \frac{\epsilon^2({\rm exp}\lbrace \kappa\theta T \rbrace - 1)}{4\kappa\theta}
$$

### Step 2): Simulating $\int_0^T \frac{ds}{X_s}$ Given $X_T$

Let $X_t$ be given by Eq. (3), then the Laplace transform of the conditional distribution of $\int_0^t \frac{ds}{X_s}|X_t$ is

$$
E\left({\rm exp}\left\lbrace -a^* \int_0^t \frac{ds}{X_s} \ \bigg| \ X_t \right\rbrace \right) = \frac{I_{\sqrt{\nu^2+8a^*/\epsilon^2}}\left(-\frac{2\kappa\theta\sqrt{X_tX_0}}{\epsilon^2{\rm sinh}\left(-\frac{\kappa\theta t}{2}\right)}\right)}{I_{\nu}\left(-\frac{2\kappa\theta\sqrt{X_tX_0}}{\epsilon^2{\rm sinh}\left(-\frac{\kappa\theta t}{2}\right)}\right)}
$$


where $I_{\nu}(z)$ denotes the modified Bessel function of the first kind, $\delta = \frac{4(\kappa + \epsilon^2)}{\epsilon^2}$ and $\nu = \frac{\delta}{2} - 1$. By setting $a^* = -ia$ we obtain the characteristic function $\Phi(a)$

The characteristic function of $\int_0^t \frac{ds}{X_s}|X_t$ is

$$
\Phi(a) = E\left({\rm exp}\left\lbrace ia \int_0^t \frac{ds}{X_s} \ \bigg| \ X_t \right\rbrace \right) = \frac{I_{\sqrt{\nu^2-8ia/\epsilon^2}}\left(-\frac{2\kappa\theta\sqrt{X_tX_0}}{\epsilon^2{\rm sinh}\left(-\frac{\kappa\theta t}{2}\right)}\right)}{I_{\nu}\left(-\frac{2\kappa\theta\sqrt{X_tX_0}}{\epsilon^2{\rm sinh}\left(-\frac{\kappa\theta t}{2}\right)}\right)}
$$

### Step 3):  Approximate the Distribution of $V_T$  $:= \frac{1}{X_T}$

Approximate $V_T$ with Log-normal distribution by matching the first and second moment $E(V_T|v_T)$, $E(V_T^2|v_T)$



### Part 2: Comparisions:

Strengths:

Compared with exact MC method, almost exact MC method is not that time-consuming. For exact MC method, the most time consuming step when sampling from the conditional distribution is the evaluation of the modified Bessel function of the first kind, $I_\nu(z)$, which has to be evaluated at **complex** $\nu$.Besides, drawing random from numerical CDF is also slow. However, for almost exact method, it supposes $V_T$ subjects to an easy and well-known distribution by matching fisrt and second moment, which saves much time.
Compared with Conditional MC, 

Baldeaux ([2012](https://doi.org/10.1142/S021902491250032X)) chooses the following set of parameters and gives the reference option price **0.443059**

$$
S_0 = 1, \quad K = 1, \quad \kappa = 2, \quad \theta = 1.5, \quad \epsilon = 0.2,
$$

$$
\rho = -0.5, \quad V_0 = 1, \quad T = 1, \quad r = 0.05
$$

In [ ]:
import pyfe
from pyfe import sv32_mc
import numpy as np

In [12]:
spot = 1
strike = np.ones(1)
mr = 2
theta_ratio = 1.5
vov = 0.2
rho = -0.5
sigma = 1
texp = 1
intr = 0.05
n_paths = 1000

In [24]:
exact = sv32_mc.ExactMC32Model(sigma, vov, rho, mr, theta_ratio, intr, divr=0.0, is_var=False, is_fwd=False)

In [25]:
exact.setExactMCparams(n_paths=1000, error=1e-5)

In [26]:
exact.price(strike, spot, texp = 1, cp_sign=1)

0.4589589871572717